In [354]:
include("utils.jl")

const kline_limit = 100 # number of candlesticks (klines) to fetch and show

coin_pair = Observable("ETCUSDT")
kline_interval = Observable("1m")
leverage = Observable(75)
stop_pct = Observable(3e-3)
profit_pct = Observable(5e-3);

price_quantity_presions = @lift get_presions($coin_pair)
price_precision = @lift $price_quantity_presions[1]
quantity_precision = @lift $price_quantity_presions[2]

Observable{Int64} with 0 listeners. Value:
2

In [355]:
function get_linesegs(Low, High, kline_limit)
    linesegs = []
    for i in 1:kline_limit
        push!(linesegs, Point2f(i, Low[i]))
        push!(linesegs, Point2f(i, High[i]))
    end
    linesegs = Point2f.(linesegs)
end

function get_ohlc(symbol, interval, limit)
    klines = get_klines(symbol, interval=interval, limit=limit)
    Open = parse.(Float64, [k[2] for k in klines])
    High = parse.(Float64, [k[3] for k in klines])
    Low = parse.(Float64, [k[4] for k in klines])
    Close = parse.(Float64, [k[5] for k in klines])
    colors = Close .>= Open

    linesegs = get_linesegs(Low, High, limit)
    last_price = Close[end]
    last_color = colors[end] ? :green : :red
    y_min = minimum(Low) * 0.997
    y_max = maximum(High) * 1.003

    Open, High, Low, Close, colors, linesegs, last_price, last_color, y_min, y_max
end

ohlcs = @lift get_ohlc($coin_pair, $kline_interval, kline_limit)

Open = @lift $ohlcs[1]
High = @lift $ohlcs[2]
Low = @lift $ohlcs[3]
Close = @lift $ohlcs[4]
colors = @lift $ohlcs[5]
linesegs = @lift $ohlcs[6]
last_price = @lift $ohlcs[7]
last_color = @lift $ohlcs[8]
y_min = @lift $ohlcs[9]
y_max = @lift $ohlcs[10];

ACCI = @lift acci($Close);

cmap = [:red, :green]

# true_leverage = @lift change_leverage(user, $coin_pair, $leverage);

2-element Vector{Symbol}:
 :red
 :green

In [356]:
user = User("api_key.json");
account_info = get_account_info(user)
change_leverage(user, coin_pair[], 75)
change_margin_mode(user, coin_pair[], "ISOLATED")
change_multi_assets_mode(user, "single")
change_position_mode(user, "hedge")

In [384]:
position = Observable(get_current_position(user, coin_pair[]))
position_amount = @lift isempty($position) ? 0.0 : round(string2float($position["positionAmt"]), digits=$quantity_precision)
liq_price = @lift isempty($position) ? 0.0 : round(string2float($position["liquidationPrice"]), digits=$price_precision)
unPnL = @lift isempty($position) ? 0.0 : round(string2float($position["unRealizedProfit"]), digits=3)

Observable{Float64} with 0 listeners. Value:
-0.008

In [330]:
fig = Figure(font="sans", fontsize=20)
vratio = 4
hratio = 4

ax1 = Axis(fig[1:vratio, 1:hratio], title=coin_pair, yaxisposition=:right)
ax2 = Axis(fig[vratio+1, 1:hratio], title="CCI", yaxisposition=:right)
deactivate_interaction!(ax1, :rectanglezoom)
deactivate_interaction!(ax2, :rectanglezoom)


barplot!(ax1, 1:kline_limit, Open, fillto=Close,
    color=colors, strokewidth=0.5, strokecolor=colors, colormap=cmap)
linesegments!(ax1, linesegs, color=colors, colormap=cmap)
hlines!(ax1, last_price, color=last_color, linestyle=:dash, linewidth=1)

# choose Long or Short mode
long_or_short = Observable("LONG")
text_long_or_short = @lift $long_or_short * " mode"
text_long_or_short_color = @lift $long_or_short == "LONG" ? :green : :red

on(events(ax1).keyboardbutton) do event
    if ispressed(fig, Keyboard.s)
        long_or_short[] = "SHORT"
    elseif ispressed(fig, Keyboard.l)
        long_or_short[] = "LONG"
    end
end

# show prices along with the mouse movements
mouse_pos = lift(events(ax1).mouseposition) do mp
    mouseposition(ax1.scene)
end

mouse_pos_x = @lift $mouse_pos[1]
mouse_open_price = @lift $mouse_pos[2]
mouse_stop_price = @lift $long_or_short == "LONG" ? $mouse_open_price * (1 - $stop_pct) : $mouse_open_price * (1 + $stop_pct)
mouse_profit_price = @lift $long_or_short == "LONG" ? $mouse_open_price * (1 + $profit_pct) : $mouse_open_price * (1 - $profit_pct)

vlines!(ax1, mouse_pos_x, color=:red, linestyle=:dash, linewidth=1)
hlines!(ax1, mouse_open_price, linestyle=:dash, linewidth=1, color=:black)
hlines!(ax1, mouse_stop_price, linestyle=:dash, linewidth=1, color=:red)
hlines!(ax1, mouse_profit_price, linestyle=:dash, linewidth=1, color=:green)

mouse_text_coor = 30
mouse_open_text = text!(ax1, mouse_text_coor, mouse_open_price, text=long_or_short, color=:black)
mouse_stop_text = text!(ax1, mouse_text_coor, mouse_stop_price, text="Stop", color=:red)
mouse_profit_text = text!(ax1, mouse_text_coor, mouse_profit_price, text="Profit", color=:green)

x_min = -3
x_max = 120
scatter!(ax1, mouse_pos; markersize=10, color=:red)
xlims!(ax1, x_min, x_max)
ylims!(ax1, y_min[], y_max[])

menu_symbol = Menu(fig, options=["BTCUSDT", "EOSUSDT", "ETHUSDT", "ETCUSDT"], default=coin_pair[])
menu_kline_interval = Menu(fig, options=["1m", "3m", "5m", "15m", "1h", "4h"], default=kline_interval[])
menu_leverage = Menu(fig, options=zip(["10", "20", "30", "50", "75", "100"], [10, 20, 30, 50, 75, 100]), default=string(leverage[]))
menu_stop = Menu(fig, options=zip(["1", "2", "3", "4", "5", "10"], [1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 1e-2]), default=string(Int(stop_pct[] * 1000)))
menu_profit = Menu(fig, options=zip(["1", "2", "3", "4", "5", "10"], [1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 1e-2]), default=string(Int(profit_pct[] * 1000)))

balances = Observable(get_balance(user, "USDT"))

wallet_balance = @lift round($balances[1], digits=2)
text_balance = @lift "Blance = " * string($wallet_balance) * "USDT"

unPnL = @lift round($balances[4], digits=2)
text_unPnL = @lift "UnPnL = " * string($unPnL) * "USDT"

fig[1:vratio+1, hratio+1] = vgrid!(
    Label(fig, "Symbol", width=nothing), menu_symbol,
    Label(fig, "TimeInterval", width=nothing), menu_kline_interval,
    Label(fig, "Leverage", width=nothing), menu_leverage,
    Label(fig, "Stop ‰", width=nothing), menu_stop,
    Label(fig, "Profit ‰", width=nothing), menu_profit,
    Label(fig, text_long_or_short, width=nothing, color=text_long_or_short_color),
    Label(fig, text_balance, width=nothing),
    Label(fig, text_unPnL, width=nothing),
    Label(fig, "PAmount", width=nothing),
    Label(fig, "PPrice", width=nothing),
    Label(fig, "OpenOrder", width=nothing),
    Label(fig, "SLOrder", width=nothing),
    Label(fig, "TPOrder", width=nothing);
    tellheight=false)



on(menu_symbol.selection) do s
    coin_pair[] = s
    ylims!(ax1, y_min[], y_max[])
end

on(menu_leverage.selection) do s
    leverage[] = s
    change_leverage(user, coin_pair[], leverage[])
end

on(menu_stop.selection) do s
    stop_pct[] = s
end

on(menu_profit.selection) do s
    profit_pct[] = s
end

on(menu_kline_interval.selection) do s
    kline_interval[] = s
    ylims!(ax1, y_min[], y_max[])
end


open_price = stop_price = profit_price = 0.0

is_open_order = false
text_coor = 102 # the x-coordinate where to put the text
sig_digits = 5 # number of significant digits

on(events(ax1).keyboardbutton) do event
    global is_open_order
    if ispressed(fig, Keyboard.r) & (!is_open_order)
        global open_price = round(mouse_open_price[], digits=price_precision[])
        global stop_price = round(mouse_stop_price[], digits=price_precision[]) 
        global profit_price = round(mouse_profit_price[], digits=price_precision[])

        global open_line = hlines!(ax1, open_price, color=:blue, linewidth=1)
        global open_text = text!(ax1, text_coor, open_price, text="$(long_or_short[])@$open_price")
        global stop_line = hlines!(ax1, stop_price, color=:red, linewidth=1)
        global stop_text = text!(ax1, text_coor, stop_price, text="SL@$stop_price")
        global profit_line = hlines!(ax1, profit_price, color=:green, linewidth=1)
        global profit_text = text!(ax1, text_coor, profit_price, text="TP@$profit_price")
        global is_open_order = true
    end
end

on(events(ax1).keyboardbutton) do event
    if ispressed(fig, Keyboard.c)
        delete!(ax1, open_line)
        delete!(ax1, open_text)
        delete!(ax1, stop_line)
        delete!(ax1, stop_text)
        delete!(ax1, profit_line)
        delete!(ax1, profit_text)
        global is_open_order = false
    end
end

# CCI indicator
lines!(ax2, ACCI, color=:blue)
vlines!(ax2, mouse_pos_x, color=:red, linestyle=:dash, linewidth=1)
hlines!(ax2, 0, color=:black, linestyle=:dash, linewidth=0.5)
hlines!(ax2, -100, color=:red, linewidth=0.5)
hlines!(ax2, 100, color=:red, linewidth=0.5)
ylims!(ax2, -200, 200)
linkxaxes!(ax1, ax2)

# fig[4, 4] = buttongrid = GridLayout(tellwidth=false)


# buttonlabels = ["long", "short"]

# buttons = buttongrid[1:length(buttonlabels), 1] = [Button(fig, label=l, buttoncolor_active=:blue) for l in buttonlabels]
# button_long = Button(fig, label = "do long")
# button_short = Button(fig, label = "do short")

# is_long = Observable(false)
# on(is_long) do val # Create a listener
#     println("is_long has been changed to $val")
# end

# on(buttons[1].clicks) do clicks
#     is_long[] = !is_long[]
#     if is_long[]
#         hlines!(ax, [100], color=:blue)
#         buttons[1] = Button(fig, label="long2", color=:blue)
#     end
# end

# on(buttons[2].clicks) do clicks
#     notify("short")
# end

# run = Button(fig[4, 4]; label="run", tellwidth=false)
display(fig)

GLMakie.Screen(...)

In [14]:
tt = 1
while tt < 200
    coin_pair[] = coin_pair[]
    kline_interval[] = kline_interval[]
    ylims!(ax1, y_min[], y_max[])
    sleep(0.005)
    tt += 1
end

In [495]:
# frames = 1:60
# record(fig, "video.mp4", frames; framerate=20) do i # i = frame number
#     coin_pair[] = coin_pair[]
#     kline_interval[] = kline_interval[]
#     ylims!(ax1, y_min[], y_max[])
#     sleep(0.005)
#     # any other manipulation of the figure here...
# end # for each step of this loop, a frame is recorded


"video.mp4"

In [260]:
get_open_orders(user, symbol)

1-element Vector{Any}:
 Dict{String, Any}("price" => "0", "timeInForce" => "GTC", "time" => 1661938173382, "priceProtect" => false, "orderId" => 13519195798, "executedQty" => "0", "updateTime" => 1661938173382, "clientOrderId" => "stop", "stopPrice" => "32.516", "status" => "NEW"…)

In [273]:
create_open_order_dict(symbol, side, quantity, price, positionSide) = create_order_dict(symbol, "LIMIT", side; quantity=quantity, price=price, newClientOrderId="open", positionSide=positionSide)
create_stop_order_dict(symbol, side, quantity, stopPrice, positionSide) = create_order_dict(symbol, "STOP_MARKET", side; quantity=quantity, stopPrice=stopPrice, newClientOrderId="stop", positionSide=positionSide)
create_profit_order_dict(symbol, side, quantity, price, positionSide) = create_order_dict(symbol, "profit", side; quantity=quantity, price=price, newClientOrderId="profit", positionSide=positionSide)

create_profit_order_dict (generic function with 1 method)

In [270]:
limit_open_order_dict = create_order_dict(coin_pair[], "LIMIT", "BUY";
    quantity=0.2, price=32.6, newClientOrderId="limit_open", positionSide="LONG")


stop_order_dict = create_order_dict(coin_pair[], "STOP_MARKET", "SELL";
    quantity=0.2, stopPrice=32.516, newClientOrderId="stop",
    positionSide="LONG")

profit_order_dict = create_order_dict(coin_pair[], "profit_MARKET", "SELL";
    quantity=0.2, stopPrice=32.589, newClientOrderId="profit",
    positionSide="LONG")

BUY ETCUSDT 0.2 qty @ 32.6 price.
SELL ETCUSDT 0.2 qty @ 0.0 price.
SELL ETCUSDT 0.2 qty @ 0.0 price.


Dict{String, Any} with 10 entries:
  "workingType"      => "CONTRACT_PRICE"
  "stopPrice"        => 32.589
  "side"             => "SELL"
  "newOrderRespType" => "RESULT"
  "symbol"           => "ETCUSDT"
  "priceProtect"     => "FALSE"
  "newClientOrderId" => "take_profit"
  "type"             => "TAKE_PROFIT_MARKET"
  "quantity"         => 0.2
  "positionSide"     => "LONG"

In [271]:
cancel_all_open_orders(user, symbol)

Dict{String, Any} with 2 entries:
  "msg"  => "The operation of cancel all open order is done."
  "code" => 200

In [272]:
order1 = execute_order(limit_open_order_dict, user)

Dict{String, Any} with 21 entries:
  "price"         => "32.600"
  "timeInForce"   => "GTC"
  "priceProtect"  => false
  "orderId"       => 13519527573
  "executedQty"   => "0"
  "updateTime"    => 1661939062120
  "clientOrderId" => "limit_open"
  "stopPrice"     => "0"
  "status"        => "NEW"
  "origType"      => "LIMIT"
  "avgPrice"      => "0.00000"
  "cumQty"        => "0"
  "cumQuote"      => "0"
  "reduceOnly"    => false
  "closePosition" => false
  "symbol"        => "ETCUSDT"
  "positionSide"  => "LONG"
  "origQty"       => "0.20"
  "side"          => "BUY"
  "workingType"   => "CONTRACT_PRICE"
  "type"          => "LIMIT"

In [256]:
order2 = execute_order(stop_order_dict, user)

Dict{String, Any} with 21 entries:
  "price"         => "0"
  "timeInForce"   => "GTC"
  "priceProtect"  => false
  "orderId"       => 13519195798
  "executedQty"   => "0"
  "updateTime"    => 1661938173382
  "clientOrderId" => "stop"
  "stopPrice"     => "32.516"
  "status"        => "NEW"
  "origType"      => "STOP_MARKET"
  "avgPrice"      => "0.00000"
  "cumQty"        => "0"
  "cumQuote"      => "0"
  "reduceOnly"    => true
  "closePosition" => false
  "symbol"        => "ETCUSDT"
  "positionSide"  => "LONG"
  "origQty"       => "0.20"
  "side"          => "SELL"
  "workingType"   => "CONTRACT_PRICE"
  "type"          => "STOP_MARKET"

In [257]:
order3 = execute_order(profit_order_dict, user)

Dict{String, Any} with 21 entries:
  "price"         => "0"
  "timeInForce"   => "GTC"
  "priceProtect"  => false
  "orderId"       => 13519195976
  "executedQty"   => "0"
  "updateTime"    => 1661938174414
  "clientOrderId" => "take_profit"
  "stopPrice"     => "32.589"
  "status"        => "NEW"
  "origType"      => "TAKE_PROFIT_MARKET"
  "avgPrice"      => "0.00000"
  "cumQty"        => "0"
  "cumQuote"      => "0"
  "reduceOnly"    => true
  "closePosition" => false
  "symbol"        => "ETCUSDT"
  "positionSide"  => "LONG"
  "origQty"       => "0.20"
  "side"          => "SELL"
  "workingType"   => "CONTRACT_PRICE"
  "type"          => "TAKE_PROFIT_MARKET"

In [189]:
query_order(user, coin_pair[], clientOrderId="limit_open")

Dict{String, Any} with 21 entries:
  "price"         => "32.621"
  "timeInForce"   => "GTC"
  "time"          => 1661936497263
  "priceProtect"  => false
  "orderId"       => 13518486479
  "executedQty"   => "0.20"
  "updateTime"    => 1661936532898
  "clientOrderId" => "limit_open"
  "stopPrice"     => "0"
  "status"        => "FILLED"
  "origType"      => "LIMIT"
  "avgPrice"      => "32.62100"
  "cumQuote"      => "6.52420"
  "reduceOnly"    => false
  "closePosition" => false
  "symbol"        => "ETCUSDT"
  "positionSide"  => "LONG"
  "origQty"       => "0.20"
  "side"          => "BUY"
  "workingType"   => "CONTRACT_PRICE"
  "type"          => "LIMIT"

In [195]:
query_order(user, coin_pair[], clientOrderId="stop")

Dict{String, Any} with 21 entries:
  "price"         => "0"
  "timeInForce"   => "GTC"
  "time"          => 1661936747645
  "priceProtect"  => false
  "orderId"       => 13518595313
  "executedQty"   => "0"
  "updateTime"    => 1661936747645
  "clientOrderId" => "stop"
  "stopPrice"     => "32.516"
  "status"        => "NEW"
  "origType"      => "STOP_MARKET"
  "avgPrice"      => "0.00000"
  "cumQuote"      => "0"
  "reduceOnly"    => true
  "closePosition" => false
  "symbol"        => "ETCUSDT"
  "positionSide"  => "LONG"
  "origQty"       => "0.32"
  "side"          => "SELL"
  "workingType"   => "CONTRACT_PRICE"
  "type"          => "STOP_MARKET"

In [191]:
query_order(user, coin_pair[], clientOrderId="profit")

Dict{String, Any} with 21 entries:
  "price"         => "33.999"
  "timeInForce"   => "GTC"
  "time"          => 1661936617470
  "priceProtect"  => false
  "orderId"       => 13518545068
  "executedQty"   => "0"
  "updateTime"    => 1661936617470
  "clientOrderId" => "take_profit"
  "stopPrice"     => "32.699"
  "status"        => "NEW"
  "origType"      => "TAKE_PROFIT"
  "avgPrice"      => "0.00000"
  "cumQuote"      => "0"
  "reduceOnly"    => true
  "closePosition" => false
  "symbol"        => "ETCUSDT"
  "positionSide"  => "LONG"
  "origQty"       => "0.32"
  "side"          => "SELL"
  "workingType"   => "CONTRACT_PRICE"
  "type"          => "TAKE_PROFIT"

In [200]:
cancel_all_open_orders(user, symbol)

Dict{String, Any} with 2 entries:
  "msg"  => "The operation of cancel all open order is done."
  "code" => 200

In [357]:
tt = get_position(user, symbol)

2-element Vector{Any}:
 Dict{String, Any}("isolatedMargin" => "0.00000000", "updateTime" => 1661957552320, "maxNotionalValue" => "10000", "liquidationPrice" => "0", "marginType" => "isolated", "notional" => "0", "isAutoAddMargin" => "false", "symbol" => "ETCUSDT", "positionAmt" => "0.00", "positionSide" => "LONG"…)
 Dict{String, Any}("isolatedMargin" => "0.14424838", "updateTime" => 1661958135045, "maxNotionalValue" => "10000", "liquidationPrice" => "32.58245594", "marginType" => "isolated", "notional" => "-6.41460000", "isAutoAddMargin" => "false", "symbol" => "ETCUSDT", "positionAmt" => "-0.20", "positionSide" => "SHORT"…)

In [358]:
tt[2]

Dict{String, Any} with 15 entries:
  "isolatedMargin"   => "0.14424838"
  "updateTime"       => 1661958135045
  "maxNotionalValue" => "10000"
  "liquidationPrice" => "32.58245594"
  "marginType"       => "isolated"
  "notional"         => "-6.41460000"
  "isAutoAddMargin"  => "false"
  "symbol"           => "ETCUSDT"
  "positionAmt"      => "-0.20"
  "positionSide"     => "SHORT"
  "markPrice"        => "32.07300000"
  "entryPrice"       => "32.368"
  "leverage"         => "75"
  "unRealizedProfit" => "0.05900000"
  "isolatedWallet"   => "0.08524838"

In [359]:
tt[1]

Dict{String, Any} with 15 entries:
  "isolatedMargin"   => "0.00000000"
  "updateTime"       => 1661957552320
  "maxNotionalValue" => "10000"
  "liquidationPrice" => "0"
  "marginType"       => "isolated"
  "notional"         => "0"
  "isAutoAddMargin"  => "false"
  "symbol"           => "ETCUSDT"
  "positionAmt"      => "0.00"
  "positionSide"     => "LONG"
  "markPrice"        => "32.07300000"
  "entryPrice"       => "0.0"
  "leverage"         => "75"
  "unRealizedProfit" => "0.00000000"
  "isolatedWallet"   => "0"

In [367]:
function get_current_position(user, symbol)
    position = get_position(user, symbol)
    filter(x->(string2float(x["isolatedMargin"])>0.0), position)[1]
end


get_current_position (generic function with 1 method)

Observable{String} with 0 listeners. Value:
"0.02920000"